# parse tbl out from HMM hits output

In [ ]:
HMM_SEARCH_OUT = "/Users/daffa/Documents/Work/jspp67_bioinf/results/hmm_homology/prpF_KT2440_magnoliopsida/prpF_KT2440_magnoliopsida-06-results.tbl"

# kegg scraping (pre LLM-planning)

In [6]:
import pandas as pd
from pathlib import Path

In [2]:
string = """
8071409
K01649 2-isopropylmalate synthase [EC:2.3.3.13] | (RefSeq) 2-isopropylmalate synthase A
8057865
K01641 hydroxymethylglutaryl-CoA synthase [EC:2.3.3.10] | (RefSeq) hydroxymethylglutaryl-CoA synthase isoform X1
8085635
K01649 2-isopropylmalate synthase [EC:2.3.3.13] | (RefSeq) 2-isopropylmalate synthase A
8073811
K01647 citrate synthase [EC:2.3.3.1] | (RefSeq) citrate synthase, glyoxysomal isoform X1
8077981
K01641 hydroxymethylglutaryl-CoA synthase [EC:2.3.3.10] | (RefSeq) hydroxymethylglutaryl-CoA synthase
110433071
K01641 hydroxymethylglutaryl-CoA synthase [EC:2.3.3.10] | (RefSeq) hydroxymethylglutaryl-CoA synthase-like
8071625
K01647 citrate synthase [EC:2.3.3.1] | (RefSeq) citrate synthase 4, mitochondrial
"""


In [3]:
def kegg_str_to_df(string):
    lines = [line for line in string.split('\n') if line.strip()]
    list_gene = [lines[i] for i in range(0, len(lines), 2)]
    list_annot = [lines[i] for i in range(1, len(lines), 2)]
    d = {"geneID": list_gene, "annotation": list_annot}    
    return pd.DataFrame(data=d)


In [4]:
kegg_str_to_df(string)

,geneID,annotation
0,8071409,K01649 2-isopropylmalate synthase [EC:2.3.3.13...
1,8057865,K01641 hydroxymethylglutaryl-CoA synthase [EC:...
2,8085635,K01649 2-isopropylmalate synthase [EC:2.3.3.13...
3,8073811,K01647 citrate synthase [EC:2.3.3.1] | (RefSeq...
4,8077981,K01641 hydroxymethylglutaryl-CoA synthase [EC:...
5,110433071,K01641 hydroxymethylglutaryl-CoA synthase [EC:...
6,8071625,K01647 citrate synthase [EC:2.3.3.1] | (RefSeq...


In [ ]:
KEGG_DIR = 'data/reference/KEGG'


## Extract Sorghum genes from KEGG pathway map00660
Parse KEGG FTP `.list` files to get all *Sorghum bicolor* genes in pathway **map00660** (C5-Branched dibasic acid metabolism), enriched with KO, NCBI Gene ID, and NCBI Protein ID.

In [3]:
from pathlib import Path
import pandas as pd

def parse_kegg_list(filepath, col_names=('source', 'target')):
    """Parse a KEGG two-column tab-separated .list file into a DataFrame.
    Strips namespace prefixes (e.g., 'sbi:', 'ko:', 'path:')."""
    df = pd.read_csv(filepath, sep='\t', header=None, names=col_names)
    for col in df.columns:
        df[col] = df[col].str.split(':', n=1).str[1]
    return df


In [4]:
PATHWAY = 'map00660'
ORGANISM = 'sbi'
KEGG_DIR = Path('data/reference/KEGG')

# 1. Get sorghum genes in pathway sbi00660
sbi_pathway = parse_kegg_list(KEGG_DIR / ORGANISM / f'{ORGANISM}_pathway.list', ('gene', 'pathway'))
pathway_id = PATHWAY.replace('map', ORGANISM)  # map00660 -> sbi00660
genes_in_pathway = sbi_pathway[sbi_pathway['pathway'] == pathway_id][['gene']].copy()
print(f"Found {len(genes_in_pathway)} {ORGANISM} genes in pathway {pathway_id}")
genes_in_pathway


Found 6 sbi genes in pathway sbi00660


,gene
3407,8055065
3408,8055458
3409,8056085
3410,8056510
3411,8073966
3412,8076795


In [5]:
# 2. Enrich with KO ortholog assignments
sbi_ko = parse_kegg_list(KEGG_DIR / ORGANISM / f'{ORGANISM}_ko.list', ('gene', 'ko'))
genes_in_pathway = genes_in_pathway.merge(sbi_ko, on='gene', how='left')

# 3. Enrich with NCBI Gene IDs
sbi_geneid = parse_kegg_list(KEGG_DIR / ORGANISM / f'{ORGANISM}_ncbi-geneid.list', ('gene', 'ncbi_geneid'))
genes_in_pathway = genes_in_pathway.merge(sbi_geneid, on='gene', how='left')

# 4. Enrich with NCBI Protein accessions
sbi_protid = parse_kegg_list(KEGG_DIR / ORGANISM / f'{ORGANISM}_ncbi-proteinid.list', ('gene', 'ncbi_proteinid'))
genes_in_pathway = genes_in_pathway.merge(sbi_protid, on='gene', how='left')

print(f"Final table: {len(genes_in_pathway)} rows x {len(genes_in_pathway.columns)} columns")
genes_in_pathway


Final table: 6 rows x 4 columns


,gene,ko,ncbi_geneid,ncbi_proteinid
0,8055065,K01703,8055065,XP_002453237
1,8055458,K00052,8055458,XP_002447363
2,8056085,K01652,8056085,XP_021315526
3,8056510,K01653,8056510,XP_002452423
4,8073966,K01704,8073966,XP_002452816
5,8076795,K01653,8076795,XP_002449227


In [10]:
# 5. Save to TSV
output_dir = Path('results/kegg')
output_dir.mkdir(parents=True, exist_ok=True)
output_path = output_dir / f'{ORGANISM}_{PATHWAY}_genes.tsv'
genes_in_pathway.to_csv(output_path, sep='\t', index=False)
print(f"Saved to {output_path}")


Saved to results/kegg/sbi_map00660_genes.tsv


In [22]:
def parse_input_file(file):
    """
    read pathway gene file
    output df that contains sp:gene -delimiter- accessionID
    """
    with open(file, 'r') as path_gene_f:
        lines = path_gene_f.readlines()
        lines.remove('species\tgene\tko\tncbi_geneid\tncbi_proteinid\n') # remove header the hard way
        sp_gene_col = []
        acc_col =[]
        for line in lines:
            # split tabs
            sp = line.split('\t')[0]
            geneid = line.split('\t')[1]
            kegg_ortho = line.split('\t')[2]
            prot_acc = line.split('\t')[3]
            # assign to prev defined list, then eventually dataframe
            sp_gene_col.append(':'.join((sp, geneid)))
            acc_col.append(prot_acc)

        d = {"gene": sp_gene_col, "accession":acc_col}
        df = pd.DataFrame(data=d)
    return df

In [23]:
sbi_map00660 = parse_input_file('/Users/daffa/Documents/Work/jspp67_bioinf/results/kegg/sbi_map00660_genes.tsv')

In [24]:
sbi_map00660

,gene,accession
0,sbi:8055065,8055065
1,sbi:8055458,8055458
2,sbi:8056085,8056085
3,sbi:8056510,8056510
4,sbi:8073966,8073966
5,sbi:8076795,8076795


## separate by gene ID

In [50]:
def parse_input_file(file):
    """
    read pathway gene file
    outputs df that contains sp:gene -delimiter- accessionID
    """
    df = pd.read_csv(file, sep='\t')
    return df

def separate_by_orthology(df):
    """
    Group DataFrame by ko column.
    Returns dict[str, pd.DataFrame] e.g. {"K01703": sub_df, ...}
    Rows with NaN ko are dropped.
    """
    df_clean = df.dropna(subset=['ko'])
    return {ko: sub_df.reset_index(drop=True) for ko, sub_df in df_clean.groupby('ko')}

def save_outfile_as_gene_name(sub_df, output_path):
    """
    Save a KO sub-DataFrame as a TSV file.
    Creates parent directories if they don't exist.
    """
    output_path = Path(output_path)
    output_path.parent.mkdir(parents=True, exist_ok=True)
    sub_df.to_csv(output_path, sep='\t', index=False)

In [53]:
df = parse_input_file("/Users/daffa/Documents/Work/jspp67_bioinf/results/kegg/ath-zma_map00020_genes.tsv")

In [54]:
separate_by_orthology(df)

{'K00025':   species       gene  ncbi_geneid      ko ncbi_proteinid
 0     ath  AT1G04410       839527  K00025      NP_171936
 1     ath  AT5G43330       834351  K00025      NP_199147
 2     ath  AT5G56720       835773  K00025      NP_200483
 3     zma  100280767    100280767  K00025   NP_001147160
 4     zma  103633793    103633793  K00025   XP_023156449
 5     zma  103648465    103648465  K00025   XP_020403782
 6     zma     542598       542598  K00025   NP_001105603,
 'K00026':    species       gene  ncbi_geneid      ko ncbi_proteinid
 0      ath  AT1G53240       841757  K00026      NP_564625
 1      ath  AT2G22780       816808  K00026      NP_179863
 2      ath  AT3G15020       820731  K00026      NP_188120
 3      ath  AT3G47520       823906  K00026      NP_190336
 4      ath  AT5G09660       830825  K00026      NP_196528
 5      zma  100193491    100193491  K00026   NP_001132077
 6      zma  100193663    100193663  K00026   NP_001132228
 7      zma  100193743    100193743  K00026

# parse by_ko files

In [17]:
by_ko = "/Users/daffa/Documents/Work/jspp67_bioinf/results/by_ko/K00052_kegg/K00052_kegg-01-pathway_genes_from_KEGG.txt"
outfile = "/Users/daffa/Documents/Work/jspp67_bioinf/results/by_ko/K00052_kegg/K00052_kegg-02-spgeneid.txt"

with open(by_ko, "r") as f:
    lines = f.readlines()
    lines.remove(lines[0])
    out = []
    for line in lines:
        sp = line.split('\t')[0]
        egi = line.split('\t')[2]
        new_line = ':'.join((sp, egi))
        out.append(new_line)

with open(outfile, 'w') as w:
    for line in out:
        w.write(f"{line}\n")


In [3]:
pepfile = "/Users/daffa/Documents/Work/jspp67_bioinf/data/reference/KEGG/sbi/T01086.pep"
from pathlib import Path
Path(pepfile).name
    

'T01086.pep'

In [6]:
pickle = Path('../../../results/gene_network/sbi_graph_annotated-z4_k10.pkl')

In [10]:
info = pickle.name.split('-')[1].split('.')[0]
minZ = info.split('_')[0].replace('z', '')
K = info.split('_')[1].replace('k', '')
print(K, minZ)

10 4
